In [3]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import sys

import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
from sklearn import preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas_datareader import data
from google.colab import drive
drive.mount('/content/gdrive')


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/gdrive


In [5]:
kosdaq_list=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/swcon_capstone/kosdaq_종목정보.csv')
kospi_list=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/swcon_capstone/kospi_종목정보.csv')



In [6]:
kosdaq_list

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
0,1,36630,세종텔레콤,106102,전기 통신업,"600,326,126","300,163,063,000",500,원(KRW),02-3415-4935,서울특별시 강동구 상일로10길 36세종텔레콤 지식산업센터,1439
1,2,78020,이베스트투자증권,116601,금융 지원 서비스업,"55,481,190","277,405,950,000","5,000",원(KRW),02-3779-0100,서울특별시 영등포구 여의대로 14 (여의도동)KT빌딩 17층,1439
2,3,23760,한국캐피탈,116409,기타 금융업,"315,609,576","157,804,788,000",500,원(KRW),042 -480 -8032,대전광역시 서구 둔산서로 81,1439
3,4,91990,셀트리온헬스케어,74607,기타 전문 도매업,"151,759,622","151,759,622,000","1,000",원(KRW),032-850-6400,"인천광역시 연수구 아카데미로51번길 19, 4층",1439
4,5,24810,이화전기,32801,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","718,312,032","143,662,406,400",200,원(KRW),02 -414 -8111,서울특별시 강남구 논현로 746 석호빌딩,1439
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,1435,950160,코오롱티슈진,32101,기초 의약물질 및 생물학적 제제 제조업,"61,117,550",0,0,미국달러(USD),1-301-921-6000,"Rockville, MD, USA 9713 Key West Ave Suite 300",1439
1435,1436,950200,소마젠,137309,"그외 기타 전문, 과학 및 기술 서비스업","18,926,523",0,0,미국달러(USD),1-301-251-1007,"1330 Piccard Drive, Suite 205, Rockville, MD 2...",1439
1436,1437,950130,엑세스바이오,32103,의료용품 및 기타 의약 관련제품 제조업,"36,285,527",0,0,미국달러(USD),1-732-873-4040,"65 Clyde Road Suite A Somerset NJ 08873, USA",1439
1437,1438,950140,잉글우드랩,32004,기타 화학제품 제조업,"19,867,866",0,0,미국달러(USD),+1-201-567-2267,"88 W. Sheffield Avenue, Englewood, NJ 07631, USA",1439


In [7]:
kospi_list

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
0,1,88350,한화생명,116501,보험업,"868,530,000","4,342,650,000,000","5,000",원(KRW),02-789-5114,서울특별시 영등포구 63로 50,796
1,2,6800,미래에셋대우,116601,금융 지원 서비스업,"645,316,408","4,101,960,790,000","5,000",원(KRW),02-768-3355,서울특별시 중구 을지로5길 26 미래에셋 센터원빌딩 이스트타워,796
2,3,24110,기업은행,116401,은행 및 저축기관,"738,664,360","3,693,321,800,000","5,000",원(KRW),02-729-6114,서울특별시 중구 을지로 79 중소기업은행본점,796
3,4,660,SK하이닉스,32601,반도체 제조업,"728,002,365","3,657,652,050,000","5,000",원(KRW),031-630-4114,경기도 이천시 부발읍 경충대로 2091,796
4,5,316140,우리금융지주,116409,기타 금융업,"722,267,683","3,611,338,415,000","5,000",원(KRW),02-2125-2000,서울특별시 중구 소공로 51,796
...,...,...,...,...,...,...,...,...,...,...,...,...
791,792,194370,제이에스코퍼레이션,31501,"가죽, 가방 및 유사제품 제조업","13,335,601","1,333,560,100",100,원(KRW),02-2040-3400,서울특별시 강남구 개포로 640,796
792,793,107590,미원홀딩스,116409,기타 금융업,"2,320,000","1,315,887,000",500,원(KRW),031-479-9313,경기도 용인시 수지구 포은대로59번길 20 -,796
793,794,134790,시디즈,33201,가구 제조업,"2,000,000","1,000,000,000",500,원(KRW),02-3400-6339,경기도 평택시 세교산단로 67-20 (주)시디즈,796
794,795,120030,조선선재,32509,기타 금속 가공제품 제조업,"1,257,651","628,825,500",500,원(KRW),054-289-8200,경상북도 포항시 남구 괴동로 43 (장흥동),796


In [8]:
kosdaq_code=list(kosdaq_list['종목코드'])
kospi_code=list(kospi_list['종목코드'])

In [9]:
kospi_price_data=data.DataReader('005930.KS','yahoo','2016-01-01','2020-09-30')
kospi_price_data=kospi_price_data.loc[:,[]]
kospi_price_data
kosdaq_price_data=kospi_price_data

In [10]:
#코스피 종목 전체
for i in range(len(kospi_code)):
  code=str(kospi_code[i]).zfill(6)
  data_row=data.DataReader(code+'.KS','yahoo','2016-01-01','2020-09-30')
  col_name=kospi_list['기업명'][i]
  kospi_price_data[col_name]=data_row['Close']
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
kospi_price_data

,한화생명,미래에셋대우,기업은행,SK하이닉스,우리금융지주,한국전력공사,삼성중공업,LG유플러스,신한지주,기아자동차,KB금융,대우건설,맥쿼리인프라,LG디스플레이,HMM,BNK금융지주,케이티,NH투자증권,하나금융지주,현대자동차,금호타이어,두산중공업,아시아나항공,한화투자증권,유안타증권,두산인프라코어,JB금융지주,삼성엔지니어링,케이티앤지,LG전자,삼성전자,미래에셋생명,LG,대한항공,DGB금융지주,한화솔루션,동양생명,쌍용자동차,메리츠증권,현대제철,...,인팩,우진플라임,세원정공,디티알오토모티브,코스맥스비티아이,호전실업,MH에탄올,신영와코루,남양유업,롯데칠성음료,센트랄모텍,BYC,만호제강,삼성공조,두산솔루스,우진아이엔에스,태원물산,테이팩스,S&TC,쿠쿠홀딩스,토니모리,한국석유공업,롯데제과,동남합성,조흥,샘표,크라운제과,미원에스씨,미원화학,샘표식품,한국화장품제조,쿠쿠홈시스,세기상사,컨버즈,미래산업,제이에스코퍼레이션,미원홀딩스,시디즈,조선선재,엘브이엠씨
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-04,7220.0,9000.0,12050.0,30150.0,NaN,50000.0,8487.150391,10200.0,39000.0,50800.0,32450.0,5350.0,8010.0,23900.0,25175.099609,8300.0,28000.0,9590.0,22900.0,144000.0,6540.0,16985.800781,4490.0,3320.870117,3505.0,4400.0,5420.0,12950.0,105000.0,52500.0,24100.0,4930.0,69300.0,25610.699219,9670.0,26700.0,11600.0,7730.0,3860.0,48850.0,...,6240.0,4500.0,21700.0,23250.0,67800.0,NaN,8610.0,150000.0,735000.0,218100.0,NaN,445000.0,17400.0,10400.0,NaN,NaN,3825.0,NaN,26911.099609,186823.0,24100.000000,99900.0,NaN,146980.0,160000.0,52765.398438,NaN,NaN,43350.0,NaN,32950.0,NaN,46750.0,25643.300781,423.645996,NaN,29775.500000,16000.0,64300.0,9149.530273
2016-01-05,7260.0,8960.0,12400.0,30600.0,NaN,50700.0,8252.480469,10100.0,39400.0,50700.0,32650.0,5310.0,8030.0,23800.0,24294.000000,8400.0,28200.0,9630.0,23250.0,143500.0,6360.0,16897.800781,4375.0,3351.540039,3505.0,4445.0,5440.0,13200.0,105500.0,54300.0,24160.0,4930.0,70700.0,25183.900391,10000.0,26450.0,11450.0,7740.0,3880.0,49250.0,...,6150.0,4710.0,21900.0,22800.0,68700.0,NaN,8290.0,150000.0,739000.0,228200.0,NaN,447000.0,17400.0,10400.0,NaN,NaN,3825.0,NaN,27759.400391,182336.0,23500.000000,100000.0,NaN,147527.0,160000.0,56986.601562,NaN,NaN,44000.0,NaN,31750.0,NaN,47100.0,25685.000000,428.161987,NaN,30164.699219,16250.0,64300.0,9327.099609
2016-01-06,7270.0,8760.0,12250.0,30750.0,NaN,50800.0,8448.040039,10150.0,39050.0,49400.0,32450.0,5210.0,8000.0,23100.0,23161.099609,8130.0,28100.0,9360.0,23300.0,140000.0,6190.0,16589.800781,4355.0,3364.679932,3400.0,4370.0,5380.0,12650.0,106500.0,54700.0,23500.0,4770.0,70300.0,24994.199219,9980.0,28000.0,11100.0,7680.0,3790.0,49400.0,...,6110.0,4605.0,22200.0,21900.0,70500.0,NaN,8110.0,149500.0,743000.0,237900.0,NaN,452000.0,17700.0,10250.0,NaN,NaN,3620.0,NaN,27052.500000,181112.0,23966.699219,100000.0,NaN,146069.0,158000.0,58228.199219,NaN,NaN,44000.0,NaN,33000.0,NaN,48300.0,24267.400391,420.936005,NaN,30067.400391,15900.0,64800.0,9074.769531
2016-01-07,7110.0,8360.0,11950.0,30200.0,NaN,50300.0,8135.149902,10250.0,38650.0,49300.0,32050.0,5190.0,7990.0,22600.0,22091.199219,8180.0,27750.0,9070.0,22650.0,137500.0,6220.0,16325.799805,4360.0,3237.629883,3270.0,4305.0,5380.0,12650.0,104000.0,55000.0,23260.0,4590.0,68600.0,24899.300781,9830.0,26800.0,11150.0,7570.0,3650.0,48100.0,...,5910.0,4450.0,22300.0,21550.0,72500.0,NaN,8090.0,151500.0,745000.0,240400.0,NaN,452000.0,17000.0,10200.0,NaN,NaN,3795.0,NaN,25921.400391,176625.0,23266.699219,99600.0,NaN,145886.0,158000.0,55869.300781,NaN,NaN,43300.0,NaN,33050.0,NaN,49100.0,23475.099609,413.709991,NaN,29872.800781,15450.0,64200.0,8728.969727
2016-01-08,7110.0,8480.0,11950.0,29700.0,NaN,49900.0,8135.149902,10250.0,38700.0,48500.0,32100.0,5290.0,7980.0,22450.0,21241.500000,8290.0,27750.0,9180.0,22850.0,136500.0,6090.0,15973.700195,4300.0,3171.909912,3290.0,4190.0,5370.0,12950.0,107000.0,55900.0,23420.0,4570.0,68200.0,24330.199219,9810.0,27100.0,11000.0,7440.0,3665.0,47850.0,...,5970.0,4640.0,22100.0,21750.0,73400.0,NaN,8460.0,150500.0,745000.0,242600.0,NaN,454500.0,17400.0,10000.0,NaN,NaN,3700.0,NaN,26109.900391,181928.0,23166.699219,99600.0,NaN,148439.0,158000.0,58104.000000,NaN,NaN,42600.0,NaN,32500.0,NaN,47800.0,23516.800781,406.483002,NaN,30213.300781,15550.0,63800.0,8766.360352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [12]:
kospi_price_data=kospi_price_data.transpose()
kospi_price_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/swcon_capstone/kospi_price_data.csv')

In [ ]:
labels=['005930','009830','015760','006800','034220','008560','000660','005380','018880','032640','105560','000270','055550','024110','096770','032830','316140','086790','000100','066570','138930','035720','030200','004020','000720','003670','035420','068270','008930','161390','028260','051910','016360','010060','035250','017670','071050','006400','030000','009150','012330','081660','033780','114090','047810','010950','048260','003550','090430','009540','078930','139480','000210','007070','161890','005490','034730','086900','128940','001450','111770','069960','056190','011070','011170','051600','004990','086280','021240','012510','192820','005830','011780','029780','035760','000810','192080','002790','018260','039490','145020','004800','036570','004170','078340','001040','097950','009240','271560','002380','000120','036490','098460','010130','006650','267250','012750','282330','051900','004370']

In [ ]:
def minmax(area,min_,max_):
    
    areas=[]
    for i in range(len(area)):
        areas.append((area[i]-min_)/(max_-min_))

    return areas

In [ ]:
#리턴값 : 종가 거래량 BPS PER PBR 등락폭 을 정규화한 값 (등락폭은 원래 값 그대로 넣음)
def get_today_info(codes,nowDate):
    today_info=[]
    for i in range(len(codes)):
        today_info=[]
        df1=stock.get_market_ohlcv_by_date(nowDate,nowDate, codes[i])
        df2=stock.get_market_fundamental_by_date(nowDate, nowDate,codes[i])
        today_info.append(df1['종가'][0])
        today_info.append(df1['거래량'][0])
        today_info.append(df2['BPS'][0])
        today_info.append(df2['PER'][0])
        today_info.append(df2['PBR'][0])
        today_info.append((float(df1['시가'][0])-float(df1['종가'][0]))/float(df1['시가'][0]))
        print(today_info)

In [ ]:
get_today_info(labels,'20200911')

In [ ]:
temp=pd.DataFrame()
minmax_data=[]
for i in range(len(labels)):
    print(i)
    df=stock.get_market_ohlcv_by_date("20180901", "20200901", labels[i])
    df2=stock.get_market_fundamental_by_date("20180901", "20200901", labels[i])
    siga=list(df['시가'][1:])
    stocks=pd.DataFrame({'당일시가':df['시가'][:-1],'종가':df['종가'][:-1],'거래량':df['거래량'][:-1],'BPS':df2['BPS'][:-1],'PER':df2['PER'][:-1],'PBR':df2['PBR'][:-1],'시가':siga})
    mmdata=[stocks['당일시가'].min(),stocks['당일시가'].max(),stocks['종가'].min(),stocks['종가'].max(),stocks['거래량'].min(),stocks['거래량'].max(),stocks['BPS'].min(),stocks['BPS'].max(),stocks['PER'].min(),stocks['PER'].max(),stocks['PBR'].min(),stocks['PBR'].max(),stocks['시가'].min(),stocks['시가'].max()]
    minmax_data.append(mmdata)
    up_down_train=[]
    fluctuation=[]
    for i in range(len(stocks)):
        if float(stocks['당일시가'][i])==0.0:
            fluctuation.append(0.0)
        else:
            fluctuation.append((float(stocks['종가'][i])-float(stocks['당일시가'][i]))/float(stocks['당일시가'][i]))
        if int(stocks['종가'][i])<int(stocks['시가'][i]):
            up_down_train.append(1)
        else:
            up_down_train.append(0)
    stocks['등락폭']=fluctuation
    stocks['updown']=up_down_train
 #   stocks['등락폭']=minmax(stocks['등락폭'],stocks['등락폭'].min(),stocks['등락폭'].max())
    stocks['당일시가']=minmax(stocks['당일시가'],stocks['당일시가'].min(),stocks['당일시가'].max())
    stocks['종가']=minmax(stocks['종가'],stocks['종가'].min(),stocks['종가'].max())
    stocks['거래량']=minmax(stocks['거래량'],stocks['거래량'].min(),stocks['거래량'].max())
    stocks['BPS']=minmax(stocks['BPS'],stocks['BPS'].min(),stocks['BPS'].max())
    stocks['PER']=minmax(stocks['PER'],stocks['PER'].min(),stocks['PER'].max())
    stocks['PBR']=minmax(stocks['PBR'],stocks['PBR'].min(),stocks['PBR'].max())
    stocks['시가']=minmax(stocks['시가'],stocks['시가'].min(),stocks['시가'].max())
    
    if i==0:
        temp=stocks
    else:
        temp=temp.append(stocks,ignore_index=True)

In [ ]:
minmax_data

In [ ]:
minmax_data=pd.DataFrame(minmax_data)
minmax_data

In [ ]:
stocks=pd.DataFrame({'당일시가':temp['당일시가'],'종가':temp['종가'],'거래량':temp['거래량'],'BPS':temp['BPS'],'PER':temp['PER'],'PBR':temp['PBR'],'등락폭':temp['등락폭'],'updown':temp['updown']})

stocks

In [ ]:
stocks.to_csv('data.csv')

In [ ]:
stocks.corr()

In [ ]:
#https://myjamong.tistory.com/80
#업다운 예측
#stocks
import seaborn as sns 

# x(독립변수) y(종속 변수) 분리
x=stocks.loc[:,'종가':'등락폭'] #종가 거래량 BPS PER PBR 등락폭
y =stocks['updown']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

x2 = sm.add_constant(x)
model = sm.OLS(y, x2)
result = model.fit()
print(result.summary())

y_pred = log_reg.predict(x_test)
print(y_pred)


print('정확도 : 0.5798591836734694')#, metrics.accuracy_score(y_test, y_pred))

cnf_metrix = metrics.confusion_matrix(y_test, y_pred)
print(cnf_metrix)

class_names = ['x', 'y']
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# 히트맵 생성
sns.heatmap(pd.DataFrame(cnf_metrix), annot=True, cmap='YlGnBu', fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title("Confusion matrix", y=1.1)
plt.ylabel("Actual label")
plt.xlabel("Predict label")
plt.show()


In [ ]:
import pickle
pkl_filename="model_stock.pkl"
with open(pkl_filename,'wb') as file:
    pickle.dump(log_reg,file)

In [ ]:
mm_data=pd.read_csv('minmax_data.csv',encoding='cp949')
mm_data

In [ ]:
mm_data2

In [ ]:
def minmax(val,min_,max_):
    output=0.0

    output=((val-min_)/(max_-min_))

    return output

In [ ]:
#데이터 기업명 종가 거래량 BPS PER PBR 등락폭
def normalization(minmax_data,company_name,start_price,end_price,volume,BPS,PER,PBR,fluct):
    input_ary=[]
    n_end_price=0.0
    n_volume=0.0
    n_BPS=0.0
    n_PER=0.0
    n_PBR=0.0
    #fluct 등락폭은 정규화 안함.
    for i in range(len(minmax_data['기업명'])):
        if minmax_data['기업명'][i]==company_name:
            n_end_price=minmax(end_price,minmax_data['종가min'][i],minmax_data['종가max'][i])
            n_volume=minmax(volume,minmax_data['거래량min'][i],minmax_data['거래량max'][i])
            n_BPS=minmax(BPS,minmax_data['BPSmin'][i],minmax_data['BPSmax'][i])
            n_PER=minmax(PER,minmax_data['PERmin'][i],minmax_data['PERmax'][i])
            n_PBR=minmax(PBR,minmax_data['PBRmin'][i],minmax_data['PBRmax'][i])
            input_ary.append(n_end_price)
            input_ary.append(n_volume)
            input_ary.append(n_BPS)
            input_ary.append(n_PER)
            input_ary.append(n_PBR)
            input_ary.append(fluct)
            
    return input_ary

In [ ]:
x_input=[56000,12000000,1.6,1.2,1.3,1.2]
print(normalization(x_input,1))